In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

2024-11-01 17:26:44.607266: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 17:26:44.806621: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-01 17:26:47.875816: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-01 17:26:49.945123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730478411.723891   47856 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730478412.14

In [2]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [ ]:

imgs = np.load('75/images.npy')
imgs = imgs.astype('float32')
indices = np.random.permutation(imgs.shape[0])
imgs = imgs[indices]

split_1 = int(18000*0.8)
split_2 = int(18000*0.9)

train_imgs = imgs[:split_1]
val_imgs = imgs[split_1:split_2]
test_imgs = imgs[split_2:]

train_imgs = train_imgs / 255.0
test_imgs = test_imgs / 255.0
val_imgs = val_imgs / 255.0

labels = np.load('75/labels.npy')
labels = labels.astype('int32')
labels = labels[indices]
train_labels = labels[:split_1]
val_labels = labels[split_1:split_2]
test_labels = labels[split_2:]

train_imgs = train_imgs.reshape((train_imgs.shape[0], 75, 75, 1))
val_imgs = val_imgs.reshape((val_imgs.shape[0], 75, 75, 1))
test_imgs = test_imgs.reshape((test_imgs.shape[0], 75, 75, 1))

## Classification Model

In [5]:

def conv_time(time):
    ntime = 0
    if time[1] > 30:
        ntime = (time[0] + 0.5)
    else:
        ntime = time[0]
    return ntime

train_labels_converted = np.array([conv_time(time) for time in train_labels])
test_labels_converted = np.array([conv_time(time) for time in test_labels])
val_labels_converted = np.array([conv_time(time) for time in val_labels])

encoder = LabelEncoder()
test_labels_encoded = encoder.fit_transform(test_labels_converted.reshape(-1))
train_labels_encoded = encoder.fit_transform(train_labels_converted.reshape(-1))
val_labels_encoded = encoder.fit_transform(val_labels_converted.reshape(-1))

OHencoder = OneHotEncoder(sparse_output=False)
train_labels_oh = OHencoder.fit_transform(train_labels_encoded.reshape(-1, 1))
val_labels_oh = OHencoder.fit_transform(val_labels_encoded.reshape(-1, 1))

In [6]:
input_shape = (75, 75, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(kernel_size=(5,5), strides = (2,2), activation="relu", filters=32))
model.add(keras.layers.Conv2D(activation="relu", filters=32, kernel_size=(3,3), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=32, activation="relu"))
model.add(keras.layers.Dense(units=24, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

print(train_imgs.shape, train_labels_oh.shape)
val_labels_oh = val_labels_oh.reshape((val_labels_oh.shape[0], 24))

model.fit(train_imgs, train_labels_oh, epochs=10, batch_size=256, validation_data=(val_imgs, val_labels_oh), callbacks=[early_stop])

/vol/home/s4201388/Documents/Leiden/IDL/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


(14400, 75, 75, 1) (14400, 24)
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.0445 - loss: 3.1780 - val_accuracy: 0.0489 - val_loss: 3.1780
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.0440 - loss: 3.1779 - val_accuracy: 0.0489 - val_loss: 3.1779
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.0396 - loss: 3.1779 - val_accuracy: 0.0489 - val_loss: 3.1780
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.0428 - loss: 3.1778 - val_accuracy: 0.0489 - val_loss: 3.1777
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.0422 - loss: 3.1776 - val_accuracy: 0.0506 - val_loss: 3.1765
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.0492 - loss: 3.1745 - val_accuracy: 0.0561 - val_loss: 3.1620
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.0595 - loss: 3.1476 - val_accuracy: 0.0772 - val_loss: 3.0729
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.0714 - loss: 3.

In [7]:
preds = model.predict(test_imgs)
preds = np.argmax(preds, axis=1)

results = encoder.inverse_transform(preds)

accuracy = np.sum(results == test_labels_converted) / len(test_labels_converted)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
9.555555555555555 %


## Regression Model

In [20]:
import tensorflow.keras.backend as K

#returning the common sense difference between two times
def difference_func(true, pred):
    true = K.cast(true, 'float32')
    diff_1 = K.abs(true - pred)
    diff_2 = K.abs(true - (pred + 12*60))

    return K.minimum(diff_1, diff_2)

In [37]:
train_labels_reg = np.array([(time[0]*60+time[1]) for time in train_labels])
test_labels_reg = np.array([(time[0]*60+time[1]) for time in test_labels])
val_labels_reg = np.array([(time[0]*60+time[1]) for time in val_labels])


model = keras.models.Sequential()
model.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), strides = (2,2),input_shape=(75, 75, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=1, activation="softplus"))
model.compile(loss=difference_func, optimizer="adam", metrics=[difference_func])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, train_labels_reg, epochs=40, batch_size = 512, validation_data = (val_imgs, val_labels_reg), callbacks = [early_stop])

Epoch 1/40


/vol/home/s4201388/Documents/Leiden/IDL/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - difference_func: 180.9812 - loss: 180.9812 - val_difference_func: 182.6678 - val_loss: 182.6678
Epoch 2/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - difference_func: 179.9021 - loss: 179.9021 - val_difference_func: 182.6679 - val_loss: 182.6679
Epoch 3/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - difference_func: 180.5134 - loss: 180.5133 - val_difference_func: 182.6681 - val_loss: 182.6681
Epoch 4/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - difference_func: 180.2920 - loss: 180.2920 - val_difference_func: 182.6682 - val_loss: 182.6682
Epoch 5/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - difference_func: 180.8344 - loss: 180.8343 - val_difference_func: 182.6682 - val_loss: 182.6682
Epoch 6/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - difference_func: 180.2010 - loss: 180.2010 - val_difference_func: 182.6681 - val_loss: 182.6681
Epoch 7/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - difference_func: 181.4108 - loss: 181.4108 - val_difference_func

In [38]:
reg_preds = model.predict(test_imgs)
accuracy = np.mean(np.abs(reg_preds - test_labels_reg) < 5)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1.0555555555555556 %


## Two-Headed Model